<h1 align="center">Práctica 4. El problema del ensamblaje (Ramificación y Poda)</h1>
<h3 style="display:block; margin-top:5px;" align="center">Algorítmica</h3>
<h3 style="display:block; margin-top:5px;" align="center">Grado en Ciencia de Datos</h3>
<h3 style="display:block; margin-top:5px;" align="center">2024-2025</h3>    
<h3 style="display:block; margin-top:5px;" align="center">Universitat Politècnica de València</h3>
<br>

**Pon/poned aquí tú/vuestros nombre(s):**
- Jaime Ballester Solá
- Marcos Ranchal García

## Índice
1. ### [El problema del ensamblaje](#introduccion)
1. ### [Actividad 1: Solución voraz](#actividad1)
1. ### [Actividad 2: Ramificación](#actividad2)
1. ### [Actividad 3: Calcular estadísticas](#actividad3)
1. ### [Código a completar](#codigo)


<a id='introduccion'></a>
# El problema del ensamblaje

Se trata del problema descrito en los apuntes de teoría en la sección 9.4. Lo mejor es ir directamente al pdf para ver la descripción del problema, pero resumimos aquí algunos datos:

- Hay que ensamblar un total de $M$ piezas con el menor coste posible.
- El coste de ensamblar la pieza $i$ depende del número de piezas ya ensambladas.
- Los datos de entrada se resumen en una matriz `costes` de tamaño $M \times M$ con valores positivos (no hace falta que sean enteros). El valor `costes[i,j]` representa el coste de situar la pieza `i` (un identificador entre `0` y `M-1`) cuando ya se han ensamblado `j` piezas.
- Las soluciones son tuplas de la forma $(x_0,x_1,\ldots,x_{M-1})$ donde $x_i$ es el nº piezas ya montadas en el momento en que se decide montar la pieza que identificamos con el índice $i$.
- La función objetivo es: $f((x_0,x_1,\ldots,x_{M-1})) = \sum_{0 \leq i < M} \mbox{costes}[i,x_i]$
- Todas las permutaciones serían factibles, se trata de encontrar una que corresponda a un coste mínimo (podría haber empates).
- Se trata de un problema conocido en teoría de grafos, el [Problema de la asignación](https://es.wikipedia.org/wiki/Problema_de_la_asignaci%C3%B3n) o [Assignment problem](https://en.wikipedia.org/wiki/Assignment_problem) para el que existen algoritmos como [Kuhn Munkres](https://en.wikipedia.org/wiki/Hungarian_algorithm) con un coste polinómico ($O(|V|^3)$). Sería interesante comparar los algoritmos de ramificación y poda de esta práctica con estos otros, pero en una sola sesión no da tiempo.

## Generación de instancias

Para generar instancias concretas para una talla dada, vamos a recurrir a la generación de números aleatorios utilizando la siguiente función de la biblioteca `numpy`:

In [1]:
import numpy as np
def genera_instancia(M, low=1, high=1000):
    return np.random.randint(low=low,high=high,
                             size=(M,M),dtype=int)

In [2]:
costes = genera_instancia(4,high=10)
costes

array([[4, 2, 2, 4],
       [1, 1, 1, 3],
       [2, 6, 9, 1],
       [1, 5, 6, 1]], dtype=int32)

Con una matriz como ésta (cada vez que lo ejecutes dará normalmente otra distinta):

```python
array([[7, 3, 7, 2],
       [9, 9, 4, 1],
       [9, 4, 8, 1],
       [3, 4, 8, 4]])
```

el coste de ensamblar la pieza 0 en la cuarta posición (después de haber ensamblado 3 piezas) es `costes[0,3]` que vale `2`.


## Cota optimista

Vamos a utilizar como cota optimista (cota inferior en este caso, pues es un problema de minimización) la suma de:

- La parte conocida.
- Una estimación optimista del coste de situar las piezas que faltan.

Concretamente, la estimación utilizada es una de las descritas en los apuntes de teoría que permite su cálculo de forma incremental:

> Suponer que para cada pieza que queda por ensamblar se selecciona la posición en la que cueste menos ensamblar, **sin importar que esa posición ya haya sido utilizada.**.


$\mbox{optimistic}((x_0,x_1,\ldots,x_{k-1})) = \sum_{0\leq i<k} \mbox{costes}[i,x_i] + \sum_{k  \leq i < M} \mbox{min}_{0 \leq j < M} \mbox{costes}[i,j] $

Es decir, completamos la parte desconocida con el menor coste posible para cada una de las piezas que quedan por ensamblar.

Obviamente, podemos precalcular esos mínimos al inicio para reutilizarlos a lo largo del algoritmo.

Esta cota se puede actualizar **de forma incremental** (teniendo en cuenta el valor de la cota del padre) al ramificar un estado.

> **Nota:** Esta cota **NO** es la más informada para podar por cota optimista. Existe otra más informada consistente en no tener en cuenta los instantes utilizados por las piezas que ya forman parte de la solución parcial. El problema de esta cota alternativa es que resulta más cara de calcular.


## Representación de los estados y del conjunto de estados activos

- Un estado intermedio $(x_0,x_1,\ldots,x_{k-1})$ se representará mediante una lista Python con esos mismos valores. El estado inicial será la lista vacía `[]`. Un estado solución será una lista de talla $M$.
- El conjunto de estados activos será una cola de prioridad implementada mediante un *minheap* similar a los estudiados en la asignatura EDA, pero vamos a recurrir a una biblioteca estándar de python llamada `heapq`:

In [3]:
import heapq

El siguiente código ilustra las 2 funciones básicas para utilizar un `heapq` como cola de prioridad.

Observa que guardamos tuplas donde el primer campo es el *score* para que así se ordenen por *score* de menor a mayor (nos sirve porque estamos en un problema de **minimización**):

In [4]:
A = [] # conjunto vacío de estados activos, es una lista Python normal y corriente
for score,s in [(10,[1]),(3,[2]),(100,[0,1])]:
    heapq.heappush(A,(score,s)) # insertar i en la cola de prioridad A
print(A) # no sale ordenado necesariamente, es un minheap...
print("Vamos sacando tupla (score,estado) ordenadamente:")
while len(A)>0:
    score,s = heapq.heappop(A) # extraer el menor elemento de la cola de prioridad A
    print(score,s)

[(3, [2]), (10, [1]), (100, [0, 1])]
Vamos sacando tupla (score,estado) ordenadamente:
3 [2]
10 [1]
100 [0, 1]


## Esquema de ramificación y poda, funciones auxiliares

Podemos implementar el esquema de ramificación y poda o *branch and bound* de dos formas alternativas e igualmente válidas (entre otras formas más):

- Utilizar una clase y métodos (como en los apuntes de teoría).
- Utilizar una función y poner dentro otras funciones (clausuras o *closure* en inglés).

Vamos a optar por esta segunda opción ya que básicamente podemos ver el problema del ensamblaje como una función que recibe la matriz de costes y nos devuelve la mejor solución encontrada (es decir, tiene un punto de entrada y uno de salida).

Las funciones auxiliares utilizadas son:

- `branch` recibe una solución parcial (un estado intermedio) y su score asociado y va generando soluciones hijas (y su respectivos score) consistentes en añadir un nuevo $x_k$. Se trata de determinar cuántas piezas se han ensamblado en el momento de ensamblar la pieza $k$-ésima. El score asociado **se debe calcular de manera incremental**.

- `is_complete` se limita a decir si una solución parcial es solución o estado terminal. Se proporciona ya implementada puesto que es tan fácil como ver si la longitud de la solución parcial es igual a $M$.

- `greedy_solution` calcula una solución *arbitraria* para inicializar la mejor solución en curso. Esto es importante para empezar a podar tan pronto como sea posible. Aunque serviría "cualquier" solución (ejemplo: `[0,1,2,...,M-1]`) cuanto mejor sea esta primera solución, tanto mejor. La restricción es que no debe ser demasiado costosa de calcular. Veremos varias formas de implementar una solución voraz.

El bucle principal de ramificación y poda es como sigue:


```python
    ...
    while len(A)>0 and A[0][0] < fx:
        s_score, s = heapq.heappop(A)
        for child_score, child in branch(s_score, s):
            if is_complete(child): # si es terminal
                # es factible (pq branch solo genera factibles)
                # falta ver si mejora la mejor solucion en curso
                if child_score < fx:
                    fx, x = child_score, child
            else: # no es terminal
                # lo metemos en el cjt de estados activos si
                # supera la poda por cota optimista:
                if child_score < fx:
                    heapq.heappush(A, (child_score, child) )
    return x,fx
```

Observa que se trata de la estrategia conocida como **poda implícita**:

- Cuando se encuentra una solución que mejora a la mejor hasta el momento se actualiza la mejor solución pero no se revisa el conjunto de estados activos `A`. Es decir, `A` puede contener soluciones que se podrían eliminar con una poda pero que cuando se introdujeron en su momento no se podaron porque el rasero o criterio era distinto. La poda explícita (no la utilizamos en esta práctica) aprovecharía este momento para eliminar esas soluciones podables.

- Por otra parte, no basta con poner `while len(A)>0` sino que hace falta añadir `and A[0][0] < fx:` porque incluso sin vaciar el conjunto de estados activos si sacamos una solución peor que `fx` (score de la mejor solución hasta el momento) todas las que queden en `A` también son podables y no tiene interés procesarlas. Esto NO haría falta con poda explícita.

A continuación mostramos las 3 actividades a realizar y posteriormente el código donde hay que implementar o codificar esas actividades.

<a id='actividad1'></a>
# Actividad 1: Solución voraz

Se trata de obtener una solución con la que inicializar la variable `x` (mejor solución encontrada hasta el momento) y su score correspondiente `fx`.

En el problema del ensamblaje es trivial obtener una solución porque cualquier permutación de índices entre `0` y `M-1` es una solución válida. Es decir, podríamos hacer algo así:

```python
def ensamblaje(costes,
               verbosity=1):

    ...
    
    def naive_solution():
        score, solution = 0, []
        for i in range(M):
            solution.append(i)
            score += costes[i,i]
        return score,solution
```

Es decir, cada $x_i$ es igual a $i$ (los objetos se ponen en orden 0,1,2,...).

Pero cuanto mejor sea la solución inicial antes empezaremos a podar mejor, sin llegar al extremo de calcularla de forma exacta porque, en ese caso ¿para qué usar luego ramificación y poda?

En esta primera actividad debes completar la función `greedy_solution` donde existen varias opciones, entre ellas:

1. Ir por orden pieza a pieza (fila $i$ de la matriz) y elegir el valor $x_i$ (el momento de colocación de esa pieza) que resulte más barato de los que siguen disponibles. Es decir, ir fila por fila de la matriz y elegir (para esa fila) la columna menor de las columnas previamente no elegidas.
2. Ir por orden instante a instante (columna de la matriz) y elegir para cada una la pieza (fila de la matriz) que sea más barata de colocar en ese instante (de entre las piezas que queden por ensamblar).
3. Ordenar de menor a mayor todos valores de la matriz de costes recordando sus coordenadas. Después se recorre utilizando los valores que correspondan a piezas e instantes válidos (descartando el resto) hasta haber situado todas las piezas.
4. Cualquier combinación de los anteriores (se calculan soluciones con varios algoritmos y nos quedamos con la mejor solución).

Debes implementar al menos la primera, las dos siguientes se pueden implementar de forma opcional. La última es trivial (llamar a las anteriores y quedarse el mejor resultado).

<a id='actividad2'></a>
# Actividad 2: Función de ramificación

Se trata de completar la función `branch` que recibe 2 argumentos:

- El score del estado padre.
- El estado padre $[x_0,x_1,\ldots,x_{k-1}]$.

Podemos asumir que dicha lista tiene una longitud menor a `M` porque sólo se utiliza `branch` si el estado no es terminal.

Esta función puede hacer una de estas dos cosas:

- Ir devolviendo los estados hijos usando `yield`.
- Generar una lista de los estados hijos y devolver dicha lista.

Ambas aproximaciones son válidas en la medida en que ambas se pueden utilizar desde la función principal en el bucle que hemos descrito arriba y que repetimos a continuación:

```python
    ...
    while len(A)>0 and A[0][0] < fx:
        s_score, s = heapq.heappop(A)
        for child_score, child in branch(s_score, s):
            ...
```

Es importante calcular el valor `child_score` de manera **incremental** a partir del valor `s_score`. Para ello, observa que preprocesamos el mínimo coste de ensamblar cada pieza en un vector llamado `minCoste`.

<a id='actividad3'></a>
# Actividad 3: Obtención de estadísticas

En esta parte básicamente debes añadir unos contadores en el cuerpo principal de "Ramificación y Poda" para calcular las siguientes cosas:

- La variable `iterations` cuenta el nº de iteraciones del bucle `while` principal.
- La variable `maxA` contabiliza el tamaño máximo que ha llegado a alcanzar el conjunto de estados activos a lo largo de la ejecución.
- `gen_states` cuenta el número total de estados generados por `branch` a lo largo de toda la ejecución.
- `podas_opt` contabiliza el nº de podas por cota optimista (estados que se han generado, que no son terminales y que no se incluyen en el conjunto de estados activos porque no pueden dar lugar a soluciones mejores que la que tenemos hasta el momento).

Esas variables ya están declaradas y se muestra el resultado por salida estándar. Falta actualizar estas variables donde corresponda.

<a id='codigo'></a>
# Código a completar

Debes realizar las 3 actividades anteriores en la siguiente celda de código:

In [5]:
import numpy as np
import heapq

def ensamblaje(costes, verbosity=1, initial='greedy'):
    """
    costes: np.ndarray MxM de enteros >= 0
    costes[i,j] = coste de montar la pieza i en el instante j.
    Devuelve (x, fx) = (mejor permutación, su coste).
    """

    # comprobaciones básicas
    assert isinstance(costes, np.ndarray), "costes debe ser ndarray"
    assert costes.ndim == 2 and costes.shape[0] == costes.shape[1], "matriz cuadrada"
    assert np.issubdtype(costes.dtype, np.integer) and costes.min() >= 0, \
           "costes debe contener enteros no negativos"

    M = costes.shape[0]
    minCoste = [int(costes[i, :].min()) for i in range(M)]
    if verbosity > 0:
        print("minCoste:", minCoste)

    def branch(s_score, s):
        k = len(s)
        usados = set(s)
        hijos = []
        for j in range(M):
            if j not in usados:
                c = int(costes[k, j])
                bound_hijo = s_score + c - minCoste[k]
                hijos.append((bound_hijo, s + [j]))
        return hijos

    def is_complete(s):
        return len(s) == M

    def naive_solution():
        score = 0
        sol = []
        for i in range(M):
            sol.append(i)
            score += int(costes[i, i])
        return score, sol

    def greedy_solution():
        disponibles = set(range(M))
        sol = [None]*M
        score = 0
        for i in range(M):
            j_best = min(disponibles, key=lambda j: costes[i, j])
            sol[i] = j_best
            score += int(costes[i, j_best])
            disponibles.remove(j_best)
        return score, sol

    if initial == 'greedy':
        initial_solution = greedy_solution
    elif initial in ('naive', 'naif'):
        initial_solution = naive_solution
    else:
        raise ValueError(f"initial option '{initial}' not supported")

    fx, x = initial_solution()
    if verbosity > 0:
        print(f"Solución inicial ({initial}): x = {x}, coste = {fx}")

    A = []
    heapq.heappush(A, (sum(minCoste), []))

    iterations = gen_states = podas_opt = maxA = 0

    while A and A[0][0] < fx:
        iterations += 1
        maxA = max(maxA, len(A))
        bound_p, s = heapq.heappop(A)

        for child_bound, child in branch(bound_p, s):
            gen_states += 1
            if is_complete(child):
                if child_bound < fx:
                    if verbosity > 0:
                        print("MEJORAMOS", x, fx, "→", child, child_bound)
                    fx, x = child_bound, child
            else:
                if child_bound < fx:
                    heapq.heappush(A, (child_bound, child))
                else:
                    podas_opt += 1

    if verbosity > 0:
        print(f"{iterations} iteraciones, max|A|={maxA}, "
              f"estados_generados={gen_states}, estados_podados={podas_opt}")

    return x, fx


## Prueba con una matriz conocida

In [6]:
prueba = np.array([[4, 4, 5, 3],
                   [2, 8, 9, 1],
                   [6, 9, 6, 3],
                   [4, 6, 7, 7]],dtype=int)
ensamblaje(prueba)

minCoste: [3, 1, 3, 4]
Solución inicial (greedy): x = [3, 0, 2, 1], coste = 17
MEJORAMOS [3, 0, 2, 1] 17 → [1, 0, 3, 2] 16
MEJORAMOS [1, 0, 3, 2] 16 → [1, 3, 2, 0] 15
16 iteraciones, max|A|=7, estados_generados=33, estados_podados=11


([1, 3, 2, 0], 15)

Si los cambios realizados son correctos (y nosotros tampoco nos hemos equivocado por nuestra parte), el resultado de la ejecución anterior sería:

```python
minCoste: [3, 1, 3, 4]
Solución inicial (voraz): [3, 0, 2, 1] de coste 17
MEJORAMOS [3, 0, 2, 1] 17 CON [1, 0, 3, 2] 16
MEJORAMOS [1, 0, 3, 2] 16 CON [1, 3, 2, 0] 15
16 iteraciones, max|A|=7, estados_generados=33, estados_podados=11
```

y el resultado devuelto sería:

```python
([1, 3, 2, 0], 15)
```

Compara los valores reportados con la siguiente ejecución que inicializa la mejor solución con la solución *naif*:

In [7]:
ensamblaje(prueba, initial='naif')

minCoste: [3, 1, 3, 4]
Solución inicial (naif): x = [0, 1, 2, 3], coste = 25
MEJORAMOS [0, 1, 2, 3] 25 → [1, 0, 3, 2] 16
MEJORAMOS [1, 0, 3, 2] 16 → [1, 3, 2, 0] 15
16 iteraciones, max|A|=14, estados_generados=33, estados_podados=4


([1, 3, 2, 0], 15)

¿Influye la solución inicial en el número de estados generados?

## Prueba con generador de instancias

A continuación se utiliza la función `genera_instancia` definida al inicio del boletín y que nos permite generar instancias de tallas que puedes controlar para realizar experimentos y comparar el uso de la función `

In [8]:
costes = genera_instancia(M=10)
print("Con inicialización voraz:")
sol1 = ensamblaje(costes)
print(sol1)
print("\nAhora mismos datos con inicialización naif:")
sol2 = ensamblaje(costes, initial='naif')
print(sol2)

# nota: sol1 no necesariamente ha de coincidir con sol2 (podría haber empates), pero el coste asociado sí

Con inicialización voraz:
minCoste: [116, 8, 32, 105, 62, 251, 137, 51, 68, 49]
Solución inicial (greedy): x = [5, 6, 7, 1, 2, 9, 4, 8, 0, 3], coste = 2301
MEJORAMOS [5, 6, 7, 1, 2, 9, 4, 8, 0, 3] 2301 → [1, 6, 7, 3, 2, 9, 4, 5, 0, 8] 1479
MEJORAMOS [1, 6, 7, 3, 2, 9, 4, 5, 0, 8] 1479 → [1, 8, 7, 3, 2, 6, 4, 5, 0, 9] 1243
575 iteraciones, max|A|=684, estados_generados=2380, estados_podados=1096
([1, 8, 7, 3, 2, 6, 4, 5, 0, 9], 1243)

Ahora mismos datos con inicialización naif:
minCoste: [116, 8, 32, 105, 62, 251, 137, 51, 68, 49]
Solución inicial (naif): x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], coste = 2053
MEJORAMOS [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 2053 → [1, 6, 7, 3, 2, 9, 4, 5, 0, 8] 1479
MEJORAMOS [1, 6, 7, 3, 2, 9, 4, 5, 0, 8] 1479 → [1, 8, 7, 3, 2, 6, 4, 5, 0, 9] 1243
575 iteraciones, max|A|=684, estados_generados=2380, estados_podados=1096
([1, 8, 7, 3, 2, 6, 4, 5, 0, 9], 1243)
